In [1]:
import pickle
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from sklearn.metrics import accuracy_score
from scipy.special import expit
from scipy.optimize import minimize_scalar
import copy
from numpy import ma
import numpy.linalg
import cupy.linalg
import time

LOAD_FROM_PICKLE = True
USE_SHRUNK_DATASET = True
USE_GPU = True

np.set_printoptions(precision=2, suppress=True)

In [2]:
if USE_GPU:
    xp = cp
else:
    xp = np

In [3]:
# load the data
if LOAD_FROM_PICKLE:
    with open('../Data/Pickle/cover_data.pickle', 'rb') as handle:
        data = xp.load(handle, allow_pickle=True)

    print('Loaded data from pickle')
else:
    data = xp.loadtxt('../Data/Cov_Type/covtype.data', delimiter=',')
    with open('../Data/Pickle/cover_data.pickle', 'wb') as handle:
        xp.save(handle, data, allow_pickle=True)

In [ ]:
print('Data shape: {}'.format(data.shape))

# used for faster testing
if USE_SHRUNK_DATASET:
    # get the number of samples for the class witht the least samples
    min_samples = xp.bincount(data[:, -1].astype(int), minlength=7)[1:].min()

    # get min number of samples for each class
    data = xp.concatenate([data[data[:, -1] == i][:min_samples] for i in range(1, 8)])

    print('New data shape:', data.shape)

# split the data into features and labels
X = data[:, :-1]
y = data[:, -1]

# normalize the features
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0) + .0000001)


In [ ]:
print('X.shape =', X.shape)
print('y.shape =', y.shape)

In [ ]:
if USE_GPU:
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, stratify=y.get())
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, stratify=y)

In [ ]:
%%time

lr_sk = SKLogisticRegression(solver='liblinear') # all params default

if USE_GPU:
    lr_sk.fit(X_train.get(), y_train.get())
    yhat = lr_sk.predict(X_test.get())
    print('Accuracy of: ',accuracy_score(y_test.get(), yhat))
else:
    lr_sk.fit(X_train, y_train)
    yhat = lr_sk.predict(X_test)
    print('Accuracy of: ',accuracy_score(y_test, yhat))
    

In [ ]:
# helper accuracy function
def print_accuracy(y, yhat):
    if USE_GPU:
        print('Accuracy of: ', round(accuracy_score(y_test.get(), yhat.get()) * 100, 3) , '%')
    else:
        print('Accuracy of: ', round(accuracy_score(y_test, yhat) * 100 , 3), '%')

def get_accuracy(y, yhat):
    if USE_GPU:
        return round(accuracy_score(y_test.get(), yhat.get()) * 100, 3)
    else:
        return round(accuracy_score(y_test, yhat) * 100 , 3)

In [ ]:
class BinaryLogisticRegression:
    # private:
    def __init__(self, eta, solver='base', iterations=20, C1=0.0, C2=0.0, line_iters=0, batch_size=0):
        self.eta = eta
        self.solver = solver
        self.iters = iterations
        self.C1 = C1
        self.C2 = C2
        self.line_iters = line_iters
        self.batch_size = batch_size
        # internally we will store the weights as self.w_ to keep with sklearn conventions

        solvers = ['base', 'line_search', 'stochastic', 'mini_batch', 'newton']
        if solver not in solvers:
            raise ValueError('solver %s is not one of %s' % (solver, solvers))
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    @staticmethod
    def _add_bias(X):
        return xp.hstack((xp.ones((X.shape[0], 1)), X)) # add bias term

    # this defines the function with the first input to be optimized
    # therefore eta will be optimized, with all inputs constant
    @staticmethod
    def _line_search_objective_function(eta, X, y, w, grad):
        wnew = w - grad * eta
        g = expit(X @ wnew)

        if USE_GPU:
            g = g.get()
            y = y.get()

        # has to be run on the CPU because of the use of the ma module
        return -np.sum(ma.log(g[y == 1])) - ma.sum(np.log(1 - g[y == 0]))

    def _add_regularization(self, grad):
        L1 = self.C1 * xp.sign(self.w_[1:])
        L2 = self.C2 -2 * self.w_[1:]
        grad[1:] += L1 + L2

        return grad

    def _get_gradient(self, X, y):
        match self.solver:
            case 'base' | 'line_search':
                ydiff = y - self.predict_proba(X, add_bias=False).ravel() # get y difference
                # scale ydiff by ratio of positive to negative samples
                # ydiff *= (y == 1).sum() / (y == 0).sum() * 0.5
                gradient = xp.mean(X * ydiff[:, xp.newaxis], axis=0) # make ydiff a column vector and multiply through
            case 'stochastic':
                idx = int(np.random.rand() * len(y)) # grab random instance
                ydiff = y[idx] - self.predict_proba(X[idx], add_bias=False) # get y difference
                gradient = X[idx] * ydiff[:, xp.newaxis] # make ydiff a column vector and multiply through
            case 'mini_batch':
                idx = np.random.choice(len(y), size=self.batch_size, replace=False) # grab random instance
                ydiff = y[idx] - self.predict_proba(X[idx], add_bias=False).ravel() # get y difference
                gradient = xp.mean(X[idx] * ydiff[:, xp.newaxis], axis=0) # make ydiff a column vector and multiply through
            case 'newton':
                g = self.predict_proba(X, add_bias=False).ravel() # get sigmoid value for all classes
                
                # the hessian has no L1 regularization and L2 will only be included if C2 is not 0
                hessian = (X * (g * (1 - g))[:, xp.newaxis]).T @ X - (2 * self.C2) # calculate the hessian

                # I swapped X.T @ np.diag(g*(1-g)) with (X * (g*(1-g))[:, xp.newaxis]).T
                # They work the same but the latter does the diagonal multiplication using way less memory
                # The reduction in memory allows the use of the GPU for the hessian calculation
                # otherwise my GPU runs out of memory

                ydiff = y - g # get y difference
                gradient = xp.sum(X * ydiff[:, np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape) # make gradient a column vector
        gradient = self._add_regularization(gradient)

        # the hessian is special
        if self.solver == 'newton':
            return xp.linalg.pinv(hessian) @ gradient
        else:
            return gradient
    
    # public:
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = xp.zeros((num_features, 1)) # init weight vector to zeros
        
        match self.solver:
            case 'base' | 'stochastic' | 'mini_batch' | 'newton':
                # for as many as the max iterations
                for _ in range(self.iters):
                    gradient = self._get_gradient(Xb, y)
                    self.w_ += gradient * self.eta # multiply by learning rate

            case 'line_search':
                for _ in range(self.iters):
                    gradient = -self._get_gradient(Xb, y)
                    # minimization inopposite direction
                    
                    # do line search in gradient direction, using scipy function
                    opts = {'maxiter':self.line_iters} # unclear exactly what this should be
                    res = minimize_scalar(self._line_search_objective_function, # objective function to optimize
                                        bounds=(0,self.eta * 10), #bounds to optimize
                                        args=(Xb, y, self.w_, gradient), # additional argument for objective function
                                        method='bounded', # bounded optimization for speed
                                        options=opts) # set max iterations
                    
                    eta = res.x # get optimal learning rate
                    self.w_ -= gradient * eta # set new function values
                    # subtract to minimize

        

    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X) > 0.5) #return the actual prediction

In [ ]:
class LogisticRegression:
    def __init__(self, eta, solver='base', iterations=20, C1=0.0, C2=0.0, line_iters=5, batch_size=5):
        self.eta = eta
        self.solver = solver
        self.iters = iterations
        self.C1 = C1
        self.C2 = C2
        self.line_iters = line_iters
        self.batch_size = batch_size
        # internally we will store the weights as self.w_ to keep with sklearn conventions

        solvers = ['base', 'line_search', 'stochastic', 'mini_batch', 'newton']
        if solver not in solvers:
            raise ValueError('solver %s is not one of %s' % (solver, solvers))
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.unique_ = xp.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        # create a classifier for each class
        for _ in range(num_unique_classes):
            blr = BinaryLogisticRegression(self.eta, self.solver, self.iters, self.C1, self.C2, self.line_iters, self.batch_size)
            self.classifiers_.append(blr)

        # print('Classifiers created')

        # train each classifier
        for blr, yval, i in zip(self.classifiers_, self.unique_, range(num_unique_classes)):
            
            y_binary = (y == yval)
            blr.fit(X, y_binary)
            # print('Classifier Fitted %d of %d' % (i + 1, num_unique_classes), end='\r')
        # print('All Classifiers Fitted        ')
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = xp.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self, X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return xp.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[xp.argmax(self.predict_proba(X), axis=1)] # take argmax along row

In [ ]:
lr = LogisticRegression(.1, solver='base', iterations=500, C1=0.01, C2=0.01)

lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print_accuracy(y_test, yhat)

In [ ]:
lr = LogisticRegression(.1, solver='line_search', iterations=100, C1=0.01, C2=0.01, line_iters=2)

lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print_accuracy(y_test, yhat)


In [ ]:
lr = LogisticRegression(.1, solver='line_search', iterations=100, C1=0.01, C2=0.01, line_iters=5)

lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print_accuracy(y_test, yhat)


In [ ]:
lr = LogisticRegression(.001, solver='stochastic', iterations=1000, C1=0.01, C2=0.01)

lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print_accuracy(y_test, yhat)

In [ ]:
lr = LogisticRegression(.1, solver='mini_batch', iterations=100, C1=0.01, C2=0.01, batch_size=2000)

lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print_accuracy(y_test, yhat)

In [ ]:
lr = LogisticRegression(.1, solver='newton', iterations=5, C1=0.01, C2=0.01)

lr.fit(X_train, y_train)
yhat = lr.predict(X_test)

print_accuracy(y_test, yhat)

In [ ]:
solvers = ['base', 'line_search', 'stochastic', 'mini_batch', 'newton']
solvers = ['newton']
base_num_iters = [100, 20, 200, 20, 1] # each solver has a different number of base iterations
iter_multipliers = [1, 5, 10]
etas = [.001, .01, .1]
Cs = [(0.0, 0.0), (0.01, 0.0), (0.0, 0.01), (0.01, 0.01)]

results = []

for solver in solvers:
    for iter_multiplier in iter_multipliers:
        for eta in etas:
            for C1, C2 in Cs:
                # time the training in milliseconds
                start = round(time.time() * 1000)

                lr = LogisticRegression(eta, solver=solver, iterations=(base_num_iters[solvers.index(solver)] * iter_multiplier), C1=C1, C2=C2)
                lr.fit(X_train, y_train)
                yhat = lr.predict(X_test)

                end = round(time.time() * 1000)

                settings = solver + ' eta: ' + str(eta) + ' C1: ' + str(C1) + ' C2: ' + str(C2) + ' iters: ' + str(base_num_iters[solvers.index(solver)] * iter_multiplier)
                accuracy = get_accuracy(y_test, yhat)
                time_taken = end - start

                # store the settings, accuracy, and time
                print(settings + ' accuracy: ' + str(accuracy) + ' time: ' + str(time_taken) + 'ms')
                results.append((settings, accuracy, time_taken))


            